In [1]:
%load_ext dotenv
%dotenv

In [2]:
from langchain_openai.chat_models import ChatOpenAI
chat = ChatOpenAI(model_name = 'gpt-4.1-mini', seed = 365, temperature=0, max_tokens = 100)

In [3]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import CommaSeparatedListOutputParser

In [4]:
list_instructions = CommaSeparatedListOutputParser().get_format_instructions()

In [5]:
print(list_instructions)

Your response should be a list of comma separated values, eg: `foo, bar, baz` or `foo,bar,baz`


In [6]:
chat_template = ChatPromptTemplate.from_messages([
    ('human', "I've recently adopted a {pet}. Could you suggest three {pet} names? \n" + list_instructions)
])

In [7]:
print(chat_template.messages[0].prompt.template)

I've recently adopted a {pet}. Could you suggest three {pet} names? 
Your response should be a list of comma separated values, eg: `foo, bar, baz` or `foo,bar,baz`


In [8]:
chain = chat_template | chat | CommaSeparatedListOutputParser()

In [9]:
result = chain.invoke({'pet': 'dog'})

In [10]:
print(result)

['Buddy', 'Luna', 'Max']


## Batching

Faster, run in parallel.

In [11]:
chain.batch([
    {'pet': 'fish'},
    {'pet': 'cat'},
    {'pet': 'tiger'}
])

[['Bubbles', 'Nemo', 'Finley'],
 ['Whiskers', 'Luna', 'Milo'],
 ['Shere Khan', 'Rajah', 'Tigris']]

## Streaming

This is more memory efficient. They are single use. Once you read them to the end, they cannot be reset to start reading again from the beginning.

In [12]:
response = chain.stream({'pet': 'leopard'})

In [14]:
for token in response:
    print(token, end = '')

['Luna']['Blaze']

In [21]:
response = chat.stream('Teach me a bit about the country Kazakhstan')

In [22]:
for token in response:
    print(token.content, end = '')

Certainly! Here’s an overview of Kazakhstan:

**Location and Geography:**  
Kazakhstan is a large country in Central Asia and is the world's ninth-largest country by land area. It is landlocked, bordered by Russia to the north, China to the east, Kyrgyzstan, Uzbekistan, and Turkmenistan to the south, and the Caspian Sea to the west. The terrain includes vast steppes, deserts, mountains, and lakes.

**Capital and Major Cities:**  
The capital city

## Piping Chains and the RunnablePassthrough Class

In [37]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables.passthrough import RunnablePassthrough

In [25]:
chat_template_tools = ChatPromptTemplate.from_template('''
What are the five most important tools a {job title} needs?
Answer only by listing the tools.
''')

chat_template_strategy = ChatPromptTemplate.from_template('''
Considering the tools provided, develop a strategy for effectively learning and mastering them:
{tools}
''')

In [32]:
string_parser = StrOutputParser()

In [33]:
chain_tools = chat_template_tools | chat | string_parser
chain_strategy = chat_template_strategy | chat | string_parser

In [35]:
print(chain_tools.invoke({'job title': 'Data scientist'}))

1. Python  
2. R  
3. SQL  
4. Jupyter Notebook  
5. Tableau


In [38]:
chain = chain_tools | { 'tools': RunnablePassthrough() } | chain_strategy

In [39]:
print(chain.invoke({'job title': 'Data scientist'}))

Certainly! Here's a structured strategy to effectively learn and master the tools you listed: Python, R, SQL, Jupyter Notebook, and Tableau. This plan balances foundational knowledge, practical application, and integration of tools.

---

### 1. **Python**

**Why:** Versatile programming language widely used in data science, automation, web development, and more.

**Strategy:**

- **Foundations:**
  - Learn syntax, data types, control structures (loops, conditionals).
  -


In [46]:
chain.get_graph().print_ascii()

     +-------------+       
     | PromptInput |       
     +-------------+       
            *              
            *              
            *              
  +--------------------+   
  | ChatPromptTemplate |   
  +--------------------+   
            *              
            *              
            *              
      +------------+       
      | ChatOpenAI |       
      +------------+       
            *              
            *              
            *              
   +-----------------+     
   | StrOutputParser |     
   +-----------------+     
            *              
            *              
            *              
+-----------------------+  
| StrOutputParserOutput |  
+-----------------------+  
            *              
            *              
            *              
     +-------------+       
     | Passthrough |       
     +-------------+       
            *              
            *              
            *       

## Runnable Parallel

This is faster than sequential processing. More than one runnable cn be involved at a time if having the same input

In [47]:
from langchain_core.runnables import RunnableParallel

In [48]:
chat_template_books = ChatPromptTemplate.from_template(
    '''
    Suggest three of the best intermediate-level {programming language} books.
    Answer only by listing the books.
    '''
)

chat_template_projects = ChatPromptTemplate.from_template(
    '''
    Suggest three interesting {programming language} projects suitable for intermediate-level programmers.
    Answer only by listing the projects.
    '''
)

In [50]:
chain_books = chat_template_books | chat | string_parser
chain_projects = chat_template_projects | chat | string_parser

In [51]:
chain_parallel = RunnableParallel({'books': chain_books, 'projects': chain_projects})

In [52]:
chain_parallel.invoke({'programming language': 'Python'})

{'books': '1. "Fluent Python" by Luciano Ramalho  \n2. "Effective Python: 90 Specific Ways to Write Better Python" by Brett Slatkin  \n3. "Python Cookbook" by David Beazley and Brian K. Jones',
 'projects': '1. Personal Expense Tracker with Data Visualization  \n2. Web Scraper for News Aggregation  \n3. Chatbot Using Natural Language Processing (NLP)'}

In [54]:
chain_parallel.get_graph().print_ascii()

            +-------------------------------+              
            | Parallel<books,projects>Input |              
            +-------------------------------+              
                   ***               ***                   
                ***                     ***                
              **                           **              
+--------------------+              +--------------------+ 
| ChatPromptTemplate |              | ChatPromptTemplate | 
+--------------------+              +--------------------+ 
           *                                   *           
           *                                   *           
           *                                   *           
    +------------+                      +------------+     
    | ChatOpenAI |                      | ChatOpenAI |     
    +------------+                      +------------+     
           *                                   *           
           *                            